# NSS Tutorial

Field	| Type
--------|-------
ukprn	| varchar(8)
institution	| varchar(100)
subject	| varchar(60)
level	| varchar(50)
question	| varchar(10)
A_STRONGLY_DISAGREE	| int(11)
A_DISAGREE	| int(11)
A_NEUTRAL	| int(11)
A_AGREE	| int(11)
A_STRONGLY_AGREE	| int(11)
A_NA	| int(11)
CI_MIN	| int(11)
score	| int(11)
CI_MAX	| int(11)
response	| int(11)
sample	| int(11)
aggregate	| char(1)

National Student Survey 2012

The National Student Survey <http://www.thestudentsurvey.com/> is presented to thousands of graduating students in UK Higher Education. The survey asks 22 questions, students can respond with STRONGLY DISAGREE, DISAGREE, NEUTRAL, AGREE or STRONGLY AGREE. The values in these columns represent PERCENTAGES of the total students who responded with that answer.

The table `nss` has one row per institution, subject and question.

In [1]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)
options(repr.matrix.max.rows=20)

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.0     v purrr   0.3.4
v tibble  3.0.1     v dplyr   0.8.5
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



Password? ·········


## 1. Check out one row

The example shows the number who responded for:

- question 1
- at 'Edinburgh Napier University'
- studying '(8) Computer Science'

**Show the the percentage who STRONGLY AGREE**

In [2]:
nss <- dbReadTable(con, 'nss')

In [3]:
nss %>% 
    filter(question=='Q01' & 
           institution=='Edinburgh Napier University' &
           subject=='(8) Computer Science') %>%
    select(A_STRONGLY_AGREE)

A_STRONGLY_AGREE
<int>
23


## 2. Calculate how many agree or strongly agree

**Show the institution and subject where the score is at least 100 for question 15.**

In [4]:
nss %>% 
    filter(question=='Q15' & score>=100) %>%
    select(institution, subject)

institution,subject
<chr>,<chr>
Kingston College,(I) Education
"Royal Holloway, University of London",(L) Geographical Studies
Solihull College,(I) Education
Stafford College,(D) Business and Administrative studies
University of Southampton,(E) Mass Communications and Documentation
University of Wolverhampton,(7) Mathematical Sciences
University of Leicester,(2) Subjects allied to Medicine
University of Newcastle upon Tyne,(E) Mass Communications and Documentation
"Bishop Grosseteste University College, Lincoln",(F) Languages


## 3. Unhappy Computer Students

**Show the institution and score where the score for '(8) Computer Science' is less than 50 for question 'Q15'**

In [5]:
nss %>% 
    filter(question=='Q15' & 
           subject=='(8) Computer Science' &
           score<50) %>%
    select(institution, score)

institution,score
<chr>,<int>
Blackburn College,45
North Lindsey College,30
Plymouth College of Art,47
Somerset College of Arts and Technology,48
"University of Wales, Newport",30
Universities of East Anglia and Essex; Joint Provision at University Campus Suffolk,45


## 4. More Computing or Creative Students?

**Show the subject and total number of students who responded to question 22 for each of the subjects '(8) Computer Science' and '(H) Creative Arts and Design'.**

> _HINT_    
> You will need to use SUM over the response column and GROUP BY subject

In [6]:
nss %>%
    filter(question=='Q22' & 
           subject %in% c(
               '(8) Computer Science', '(H) Creative Arts and Design')) %>%
    group_by(subject) %>% 
    summarise(tot_response=sum(response))

subject,tot_response
<chr>,<int>
(8) Computer Science,10252
(H) Creative Arts and Design,33336


## 5. Strongly Agree Numbers

**Show the subject and total number of students who A_STRONGLY_AGREE to question 22 for each of the subjects '(8) Computer Science' and '(H) Creative Arts and Design'.**

> _HINT_    
> The A_STRONGLY_AGREE column is a percentage. To work out the total number of students who strongly agree you must multiply this percentage by the number who responded (response) and divide by 100 - take the SUM of that.

In [7]:
nss %>% 
    filter(question=='Q22' & 
           subject %in% c(
               '(8) Computer Science', '(H) Creative Arts and Design')) %>% 
    group_by(subject, question) %>% 
    mutate(n=response * A_STRONGLY_AGREE) %>%
    summarise(n_strongly_agree=round(sum(n)/100, 0))

subject,question,n_strongly_agree
<chr>,<chr>,<dbl>
(8) Computer Science,Q22,3421
(H) Creative Arts and Design,Q22,12108


## 6. Strongly Agree, Percentage

**Show the percentage of students who A_STRONGLY_AGREE to question 22 for the subject '(8) Computer Science' show the same figure for the subject '(H) Creative Arts and Design'.**

Use the **ROUND** function to show the percentage without decimal places.

In [8]:
nss %>% 
    filter(question=='Q22' & 
           subject %in% c(
               '(8) Computer Science', '(H) Creative Arts and Design')) %>%
    group_by(subject) %>%
    summarise(pct_strongly_agree=round(sum(A_STRONGLY_AGREE * response)/sum(response)))

subject,pct_strongly_agree
<chr>,<dbl>
(8) Computer Science,33
(H) Creative Arts and Design,36


## 7. Scores for Institutions in Manchester

**Show the average scores for question 'Q22' for each institution that include 'Manchester' in the name.**

The column **score** is a percentage - you must use the method outlined above to multiply the percentage by the **response** and divide by the total response. Give your answer rounded to the nearest whole number.

In [9]:
nss %>% 
    filter(question=='Q22' & 
           str_detect(institution, 'Manchester')) %>%
    group_by(institution) %>%
    summarise(tot_score=round(sum(response * score)/sum(response)))

institution,tot_score
<chr>,<dbl>
Manchester Metropolitan University,81
The Manchester College,72
University of Manchester,83


## 8.Number of Computing Students in Manchester

**Show the institution, the total sample size and the number of computing students for institutions in Manchester for 'Q01'.**

In [10]:
nss %>% 
    filter(question=='Q22' & 
           str_detect(institution, 'Manchester')) %>%
    mutate(comp=case_when(
        subject=='(8) Computer Science' ~ sample,
        TRUE                            ~ 0L)) %>%
    group_by(institution) %>%
    summarise(tot_sample=sum(sample), comp=sum(comp))

institution,tot_sample,comp
<chr>,<int>,<int>
Manchester Metropolitan University,6994,310
The Manchester College,537,46
University of Manchester,8065,180


In [11]:
dbDisconnect(con)